In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Set MLflow tracking URI (optional - defaults to local)
mlflow.set_tracking_uri("http://localhost:5000")

# Set MLflow experiment
experiment_name = "iris_classification"
mlflow.set_experiment(experiment_name)

# Load iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Start MLflow run
with mlflow.start_run():
    # Parameters
    n_estimators = 100
    max_depth = 3
    random_state = 42
    
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)
    
    # Train model
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state
    )
    rf.fit(X_train, y_train)
    
    # Predictions
    y_pred = rf.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("train_size", len(X_train))
    mlflow.log_metric("test_size", len(X_test))
    
    # Log model
    mlflow.sklearn.log_model(rf, "random_forest_model")
    
    # Log dataset info
    mlflow.log_param("features", iris.feature_names)
    mlflow.log_param("target_names", iris.target_names.tolist())
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Experiment Name: {experiment_name}")
    print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
    print(f"MLflow Run ID: {mlflow.active_run().info.run_id}")
    print(f"Experiment ID: {mlflow.active_run().info.experiment_id}")
    print(f"MLflow UI URL: {mlflow.get_tracking_uri()}")
    print(f"Run URL: {mlflow.get_tracking_uri()}/#/experiments/{mlflow.active_run().info.experiment_id}/runs/{mlflow.active_run().info.run_id}")

# View results
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

print("\n=== Feature Importance ===")
feature_importance = pd.DataFrame({
    'feature': iris.feature_names,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)
print(feature_importance)